In [53]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error

In [66]:
len(pd.read_csv("data/test_impute_all.csv"))

25371

In [67]:
len(pd.read_csv("data/test.csv"))

61953

In [153]:
def test(df):
    df["label"] = pd.read_csv("data/test_orig_all.csv")["Crt3"]
    train = df.sample(frac=0.8, random_state=42)
    test = df.drop(train.index)
    
    y_train = train["label"].to_numpy()
    y_test = test["label"].to_numpy()
    
    train = train.drop(columns=["Crt0", "Crt1", "Crt2", "Crt3", "label", "hadm_id", "Unnamed: 0"]).to_numpy()
    test = test.drop(columns=["Crt0", "Crt1", "Crt2", "Crt3", "label", "hadm_id", "Unnamed: 0"]).to_numpy()

    reg = xgb.XGBRegressor()
    reg.fit(train, y_train)
    pred = reg.predict(test)

    x = []
    y = []
    for a, b in zip(y_test, pred):
        if a == 0:
            continue
        x.append(a)
        y.append(b)
    
    return root_mean_squared_error(y_test, pred), mean_absolute_percentage_error(x, y)

In [154]:
test(pd.read_csv("data/test_orig_all.csv"))

(0.013887552947283567, 0.36070546982261226)

In [155]:
test(pd.read_csv("data/test_impute_all.csv"))

(0.016688449111963287, 0.44261377822009274)

In [180]:
from ast import literal_eval

def reformat(df):
    o = pd.read_csv("data/test_orig_all.csv").set_index("hadm_id")
    df = df.set_index("hadm_id")
    df = df.loc[o.index]
    d = []
    
    for i, row in df.iterrows():
        r = {"hadm_id": i}
        for c in ['Bic', 'Crt', 'Pot', 'Sod', 'Ure', 'Hgb', 'Plt', 'Wbc']:
            l = literal_eval(row[c])
            for j in range(4):
                r[c + str(j)] = l[j]
        r["gender"] = o.loc[i]["gender"]
        r["age"] = o.loc[i]["age"]
        r["weight"] = o.loc[i]["weight"]
        r["Unnamed: 0"] = i
        d.append(r)
        
    return pd.DataFrame(d)

In [156]:
test(reformat(pd.read_csv("data/medianImputedData_filtered.csv")))

(0.013827568032816893, 0.3725005580903613)

In [157]:
test(reformat(pd.read_csv("data/meanImputedData_filtered.csv")))

(0.013938187817125445, 0.3765561901679022)

In [158]:
test(reformat(pd.read_csv("data/ffillImputedData_filtered.csv")))

(0.015753858220020196, 0.4272414193806784)

In [181]:
test(pd.read_csv("data/miceImputedData_filtered.csv"))

(0.014917234517318554, 0.40566447388530585)

In [184]:
df = pd.read_csv("data/bert_imputed_data.csv").set_index("hadm_id").drop(columns=["Unnamed: 0.1", "Time0", "Time1", "Time2", "Time3"])
o = pd.read_csv("data/test_orig_all.csv").set_index("hadm_id")
df["gender"] = o["gender"]
df["age"] = o["age"]
df["weight"] = o["weight"]
df = df.reset_index()
test(df)

(0.01479750284414462, 0.35406361676777887)